In [1]:
"""
Name: Michael Austin
Login-ID: maa0075
ID number: 800-259-122
Assignment #: PA-2
Date: 04/06/2019
Purpose: Implement a simplified access control platform. 
"""

import os
import sys
import platform
import owner
#import friend

"""
Constants
"""
START = 'friendadd'
NAME_LEN = 30
NAME_FORB =['/',':', '\n','\t', '\ ']
COMMANDS = ['friendadd','viewby','logout','listadd','friendlist','postpicture',
            'chlst','chmod','chown','readcomments','writecomments','end']

# Output file names 
FRIEND = 'friends.txt'
LIST = 'lists.txt'
PIC = 'pictures.txt'
AUDIT = 'audit.txt'

In [2]:

def get_file(filename):
    # Detects the OS: (Windows, Linux, MacOS = Darwin)
    this_os = platform.system()
    start = ''
    # Decides which root to start with
    if this_os == 'Windows':
        start = 'c:\\users'
    elif this_os == 'Linux':
        start = 'root\\'
    elif this_os == 'Darwin':
        start = '\\User\\'
    
    # Checks current directory for files, first.
    cwd = os.getcwd()
    if filename in cwd:
        file = os.path.abspath(cwd+'\\'+filename)
        print(file)
    else:
        """
        File not found in cwd, start from the root and work down 
        until you find the file
        """
        for root, dirs, files in os.walk(start):
            for name in files:
                if name == filename:
                    return os.path.abspath(os.path.join(root, name))
    
    

def read_text(file):
    """
    Returns each line in the commands file as a list
    """
    line_list = []
    # Standard Python open(), specifying read mode and taking in filename from above
    with open(file,'r') as f:
        for line in f:
            line_list.append(line.split())
    return line_list


def make_file(filename, first_entry):
    """  Creates the initial friends, audit, list, or pictures output files  """
    with open(filename, 'w') as f:
        f.write(first_entry)


def add_output_entry(file,text):
    with open(file, 'a') as f:
        f.write(text + "\n")

def pre_check(cmd_list):
    for i in cmd_list:
        if i[0] not in COMMANDS:
            add_output_entry(AUDIT,f'{i} is not one of the valid commands.')
            print(f'{i} is not one of the valid commands.')

"""
def main(arg):

    
    Gets the root of the current operating system for the absolute filepath
    Returns a file object
    
    file = get_file(arg)
    
    
    Each element in f is a list in which each word in the line of 
    text read in is an element. E.g. f = [[friendadd,root], [viewby, root], ...]
    
    f = read_text(file)
    print(f[0])
    

#main('TestCase1.txt')
main(sys.argv[1])
"""
    

"\ndef main(arg):\n\n    \n    Gets the root of the current operating system for the absolute filepath\n    Returns a file object\n    \n    file = get_file(arg)\n    \n    \n    Each element in f is a list in which each word in the line of \n    text read in is an element. E.g. f = [[friendadd,root], [viewby, root], ...]\n    \n    f = read_text(file)\n    print(f[0])\n    \n\n#main('TestCase1.txt')\nmain(sys.argv[1])\n"

In [4]:
try:
    file = get_file('TestCase1.txt')
    f = read_text(file)
    make_file(AUDIT, 'Open Command File.\n')
    print('Open Command File')
except FileNotFoundError as err:
    make_file(AUDIT, err+' File was not found.')
    print('File was not found')
else:
    pre_check(f)


Open Command File
